In [1]:
import replicate
from datasets import load_dataset

import json
import jsonlines

In [2]:
%cd pass@k_eval

/Users/sangmin/Desktop/Dev/Korea_Univ/short-coding-LLM/pass@k_eval


In [3]:
from getpass import getpass
import os

REPLICATE_API_TOKEN = getpass()
os.environ['REPLICATE_API_TOKEN'] = REPLICATE_API_TOKEN

In [4]:
%env REPLICATE_API_TOKEN=$REPLICATE_API_TOKEN

env: REPLICATE_API_TOKEN=r8_RMJqckxfNA4XbauF3j06uxMoJ3Y2Vsy09dIBj


In [98]:
model = replicate.models.get("meta/codellama-34b-instruct")
version = model.versions.get("b17fdb44c843000741367ae3d73e2bb710d7428a662238ddebbf4302db2b5422")

In [6]:
params = {
    "top_k": 50,
    "top_p": 1.0,
    "max_tokens": 300,
    "temperature": 0,
    "system_prompt": "Responses should be written in Python.",
    "repeat_penalty": 1.1,
    "presence_penalty": 0,
    "frequency_penalty": 0
}

# Code Summarization: 2-stage
(1) 함수를 구현하고, (2) 구현 결과를 다시 프롬프팅하여 코드를 축약

In [89]:
dataset_pred = load_dataset("json", data_files="predictions/prediction_instruct_baseline.jsonl", split="train")
dataset_pred

Found cached dataset json (/Users/sangmin/.cache/huggingface/datasets/json/default-a549b1e62565d4d7/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


Dataset({
    features: ['task_id', 'completion'],
    num_rows: 164
})

In [69]:
template = """Rewrite the code below as short as possible.
[PYTHON]
{}
[/PYTHON]"""
print(template.format(dataset_pred[123]["completion"]))

Rewrite the code below as short as possible.
[PYTHON]
def get_odd_collatz(n):
    if n <= 0:
        raise ValueError("n should be a positive integer.")
    
    collatz_list = [n]
    while n != 1:
        if n % 2 == 0:
            n //= 2
        else:
            n = 3 * n + 1
        collatz_list.append(n)
    
    return sorted([x for x in collatz_list if x % 2 != 0])
[/PYTHON]


In [99]:
template = """Your task is 
to condense Python function that must be between the [PYTHON] and [/PYTHON] tag.
Do not change the function name. Please do not repeat.
You should apply following tips:
1. Erase comments.
2. Use dictionaries.
3. Change for statements to comprehensions.
4. Change if statements to conditional expressions or comprehensions.
5. Use built-in functions.
6. Use short circuit.
7. Use "join" to print everything at once.
8. Use short variable names.
problem :
[PYTHON]
{completion}
[/PYTHON]
"""

In [100]:
dataset_pred[0]['completion']

'def has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i+1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) <= threshold:\n                return True\n    return False'

In [102]:
print(template.format(completion=dataset_pred[0]["completion"]))

Your task is 
to condense Python function that must be between the [PYTHON] and [/PYTHON] tag.
Do not change the function name. Please do not repeat.
You should apply following tips:
1. Erase comments.
2. Use dictionaries.
3. Change for statements to comprehensions.
4. Change if statements to conditional expressions or comprehensions.
5. Use built-in functions.
6. Use short circuit.
7. Use "join" to print everything at once.
8. Use short variable names.
problem :
[PYTHON]
def has_close_elements(numbers: List[float], threshold: float) -> bool:
    for i in range(len(numbers)):
        for j in range(i+1, len(numbers)):
            if abs(numbers[i] - numbers[j]) <= threshold:
                return True
    return False
[/PYTHON]



In [104]:
predictions = []
for data in dataset_pred:
    prediction = replicate.predictions.create(
        version=version,
        input={
            **params,
            "prompt": template.format(completion=data["completion"]),
        },
    )
    predictions.append(prediction)

# Code Summarization: 2-stage (1-shot)

## Try 1:

In [132]:
template = """Rewrite the code as short as possible.
[PYTHON]
{completion}
[/PYTHON]
Just refer to the example of code condensation with the delimiter #.
[PYTHON]
{example}
[/PYTHON]
"""

In [237]:
example = """#Example
##input
def minLengthAfterRemovals(self, nums: List[int]) -> int:
    counter = Counter(nums)
    heap = [ -cnt for cnt in counter.values()]
    heapify(heap)
    while len(heap) > 1:
        largest = -heappop(heap)
        second = -heappop(heap)
        if largest > 1:
            heappush(heap, 1-largest)
        if second > 1:
            heappush(heap, 1-second) 
    return -heap[0] if heap else 0
##output
def minLengthAfterRemovals(self, nums: List[int]) -> int:
		scount = Counter(nums)
		arr = sorted([scount[num] for num in sorted(scount.keys())], key=lambda x: -x)    
		
		return (arr[0] - sum(arr[1:])) if arr[0] > sum(arr[1:]) else sum(arr) % 2
"""
print(example)

#Example
##input
def minLengthAfterRemovals(self, nums: List[int]) -> int:
    counter = Counter(nums)
    heap = [ -cnt for cnt in counter.values()]
    heapify(heap)
    while len(heap) > 1:
        largest = -heappop(heap)
        second = -heappop(heap)
        if largest > 1:
            heappush(heap, 1-largest)
        if second > 1:
            heappush(heap, 1-second) 
    return -heap[0] if heap else 0
##output
def minLengthAfterRemovals(self, nums: List[int]) -> int:
		scount = Counter(nums)
		arr = sorted([scount[num] for num in sorted(scount.keys())], key=lambda x: -x)    
		
		return (arr[0] - sum(arr[1:])) if arr[0] > sum(arr[1:]) else sum(arr) % 2



## Try 2

In [241]:
template = """
Your task is to condense Python function that must be between the [PYTHON] and [\PYTHON] tag. Do not change the function name. Please do not repeat. Here is an example :
{example}
problem :
[PYTHON]
{completion}
[/PYTHON]
"""

In [270]:
example = """def minLengthAfterRemovals(self, nums: List[int]) -> int:
    counter = Counter(nums)
    heap = [ -cnt for cnt in counter.values()]
    heapify(heap)
    while len(heap) > 1:
        largest = -heappop(heap)
        second = -heappop(heap)
        if largest > 1:
            heappush(heap, 1-largest)
        if second > 1:
            heappush(heap, 1-second) 
    return -heap[0] if heap else 0
def minLengthAfterRemovals(self, nums: List[int]) -> int:
    scount = Counter(nums)
    arr = sorted([scount[num] for num in sorted(scount.keys())], key=lambda x: -x)    
    
    return (arr[0] - sum(arr[1:])) if arr[0] > sum(arr[1:]) else sum(arr) % 2"""

print(example)

def minLengthAfterRemovals(self, nums: List[int]) -> int:
    counter = Counter(nums)
    heap = [ -cnt for cnt in counter.values()]
    heapify(heap)
    while len(heap) > 1:
        largest = -heappop(heap)
        second = -heappop(heap)
        if largest > 1:
            heappush(heap, 1-largest)
        if second > 1:
            heappush(heap, 1-second) 
    return -heap[0] if heap else 0
def minLengthAfterRemovals(self, nums: List[int]) -> int:
    scount = Counter(nums)
    arr = sorted([scount[num] for num in sorted(scount.keys())], key=lambda x: -x)    
    
    return (arr[0] - sum(arr[1:])) if arr[0] > sum(arr[1:]) else sum(arr) % 2


## Try 3:

In [282]:
before = """def minLengthAfterRemovals(self, nums: List[int]) -> int:
    counter = Counter(nums)
    heap = [ -cnt for cnt in counter.values()]
    heapify(heap)
    while len(heap) > 1:
        largest = -heappop(heap)
        second = -heappop(heap)
        if largest > 1:
            heappush(heap, 1-largest)
        if second > 1:
            heappush(heap, 1-second) 
    return -heap[0] if heap else 0
"""

after = """def minLengthAfterRemovals(self, nums: List[int]) -> int:
    scount = Counter(nums)
    arr = sorted([scount[num] for num in sorted(scount.keys())], key=lambda x: -x)    
    
    return (arr[0] - sum(arr[1:])) if arr[0] > sum(arr[1:]) else sum(arr) % 2"""

example = {
    "before": before,
    "after": after
}

In [370]:
template = """Your task is to rewrite a Python function in a shorter form. The function name and parameters must be the same as before. 
Outputs must be between the [PYTHON] and [\PYTHON] tags.

You are given one example. 

Input:
{before}
Output:
{after}

Input:
{completion}

Output:
"""

In [371]:
print(template.format(before=example["before"], after=example["after"], completion=dataset_pred[-1]["completion"]))

Your task is to rewrite a Python function in a shorter form. The function name and parameters must be the same as before. 
Outputs must be between the [PYTHON] and [\PYTHON] tags.

You are given one example. 

Input:
def minLengthAfterRemovals(self, nums: List[int]) -> int:
    counter = Counter(nums)
    heap = [ -cnt for cnt in counter.values()]
    heapify(heap)
    while len(heap) > 1:
        largest = -heappop(heap)
        second = -heappop(heap)
        if largest > 1:
            heappush(heap, 1-largest)
        if second > 1:
            heappush(heap, 1-second) 
    return -heap[0] if heap else 0

Output:
def minLengthAfterRemovals(self, nums: List[int]) -> int:
    scount = Counter(nums)
    arr = sorted([scount[num] for num in sorted(scount.keys())], key=lambda x: -x)    
    
    return (arr[0] - sum(arr[1:])) if arr[0] > sum(arr[1:]) else sum(arr) % 2

Input:
def generate_integers(a, b):
    if a > b:
        a, b = b, a
    even_digits = []
    for i in range(a, b+1):


In [372]:
predictions = []
count = 0
for data in dataset_pred:
    prediction = replicate.predictions.create(
        version=version,
        input={
                **params,
                "prompt": template.format(before=example["before"], after=example["after"], completion=data["completion"])
            },
    )
    predictions.append(prediction)
    # count += 1
    # if count == 2:
        # break

# Code Summarization: 1-stage
구현할 때부터 짧게 구현해달라고 요청

In [ ]:
dataset = load_dataset("openai_humaneval", split="test")
dataset

In [34]:
template = """Implement the code below as short as possible.
[PYTHON]
{}
[/PYTHON]"""
print(template.format(dataset[0]["prompt"]))

Implement the code below as short as possible.
[PYTHON]
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """

[/PYTHON]


In [37]:
predictions = []
for example in dataset:
    prediction = replicate.predictions.create(
        version=version,
        input={
                "top_k": 50,
                "top_p": 1.0,
                "max_tokens": 500,
                "temperature": 0,
                "system_prompt": "Responses should be written in Python.",
                "prompt": template.format(example["prompt"]),
                "repeat_penalty": 1.1,
                "presence_penalty": 0,
                "frequency_penalty": 0
            },
    )
    predictions.append(prediction)

# Self-Planning

## Planning phase

In [396]:
template = """[INST] Your task is to 
Plan the steps to implement the following Python function. 
Put a number in front of each step.
Response with ONLY natural language plan steps, with numbering.
DO NOT print python function. Print ONLY plans.
[\INST]

[PYTHON]
{completion}
[\PYTHON]
"""

In [397]:
print(template.format(completion=dataset[30]["prompt"].strip()))

[INST] Your task is to 
Plan the steps to implement the following Python function. 
Put a number in front of each step.
Response with ONLY natural language plan steps, with numbering.
DO NOT print python function. Print ONLY plans.
[\INST]

[PYTHON]
def get_positive(l: list):
    """Return only positive numbers in the list.
    >>> get_positive([-1, 2, -4, 5, 6])
    [2, 5, 6]
    >>> get_positive([5, 3, -5, 2, -3, 3, 9, 0, 123, 1, -10])
    [5, 3, 2, 3, 9, 123, 1]
    """
[\PYTHON]



In [399]:
predictions = []
count = 0
for data in dataset:
    prediction = replicate.predictions.create(
        version=version,
        input={
                **params,
                "prompt": template.format(completion=data["prompt"].strip()),
                "system_prompt": "Responses should be written in natural language."
            },
    )
    predictions.append(prediction)

## Implementation phase

In [ ]:
template = """"""

In [16]:
template = """[INST] Your task is to 
Implement the following Python function using the given steps.
Response with ONLY natural language plan steps, with numbering.
The Python code must be between [PYTHON] and [/PYTHON] tags.
DO NOT print plan steps. Print ONLY python code.
Plan steps are following:
{plan}
[/INST]
[PYTHON]
{completion}
[/PYTHON]
"""

In [20]:
template = """[PYTHON] 
{completion}
{plan}
\"\"\"
[/PYTHON]
"""

In [11]:
dataset = load_dataset("openai_humaneval", split="test")
dataset

Found cached dataset openai_humaneval (/Users/sangmin/.cache/huggingface/datasets/openai_humaneval/openai_humaneval/1.0.0/2955cebd73602e828fa8c0a424c594e5fab4ec863b316ca98f3d8fdb6a626e75)


Dataset({
    features: ['task_id', 'prompt', 'canonical_solution', 'test', 'entry_point'],
    num_rows: 164
})

In [8]:
dataset_plan = load_dataset("json", data_files="predictions/prediction_planning.jsonl", split="train")
dataset_plan[0]

Found cached dataset json (/Users/sangmin/.cache/huggingface/datasets/json/default-256b9e0deafeb836/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


{'task_id': 'HumanEval/0',
 'completion': '1. Sort the list of numbers in ascending order.\n2. Iterate through the sorted list and check if any two adjacent elements are closer to each other than the given threshold.\n3. If two adjacent elements meet the condition, return True.\n4. If no two adjacent elements meet the condition, return False.'}

In [25]:
plan = "\n".join("    " + line for line in (dataset_plan[0]["completion"].split("\n")))
print(plan)

    1. Sort the list of numbers in ascending order.
    2. Iterate through the sorted list and check if any two adjacent elements are closer to each other than the given threshold.
    3. If two adjacent elements meet the condition, return True.
    4. If no two adjacent elements meet the condition, return False.


In [27]:
prompt = "\n".join(dataset[0]["prompt"].strip().split("\n")[:-1])
print(prompt)

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True


In [38]:
template = """[PYTHON] 
{completion}
{plan}
    \"\"\"
[/PYTHON]
"""

In [46]:
def format_prompt(plan, completion):
    plan = "\n".join("    " + line for line in (plan.split("\n")))
    completion = "\n".join(completion.strip().split("\n")[:-1])
    
    return template.format(plan=plan, completion=completion)

In [47]:
for i, (plan, completion) in enumerate(zip(dataset_plan, dataset)):
    print(format_prompt(plan["completion"], completion["prompt"]))
    print("-" * 50)

[PYTHON] 
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    1. Sort the list of numbers in ascending order.
    2. Iterate through the sorted list and check if any two adjacent elements are closer to each other than the given threshold.
    3. If two adjacent elements meet the condition, return True.
    4. If no two adjacent elements meet the condition, return False.
    """
[/PYTHON]

--------------------------------------------------
[PYTHON] 
from typing import List


def separate_paren_groups(paren_string: str) -> List[str]:
    """ Input to this function is a string containing multiple groups of nested parentheses. Your goal is to
    separate those group into separate strings and return the

In [50]:
predictions = []
count = 0
for i, (plan, completion) in enumerate(zip(dataset_plan, dataset)):
    prediction = replicate.predictions.create(
        version=version,
        input={
                **params,
                "prompt": format_prompt(plan["completion"], completion["prompt"]),
            },
    )
    predictions.append(prediction)

## Plannning & Impelementation

In [72]:
template = """
[INST] Your task is to 
Plan the steps to implement the following Python function. 
Next, implement the function according to the created steps. 
The Python code must be between [PYTHON] and [/PYTHON] tags.
You don't need to print plan steps.
[/INST]

[PYTHON]
{completion}
[/PYTHON]
"""

In [73]:
print(template.format(completion=dataset[0]["prompt"].strip()))


[INST] Your task is to 
Plan the steps to implement the following Python function. 
Next, implement the function according to the created steps. 
The Python code must be between [PYTHON] and [/PYTHON] tags.
You don't need to print plan steps.
[/INST]

[PYTHON]
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
[/PYTHON]



In [75]:
predictions = []
count = 0
for data in dataset:
    prediction = replicate.predictions.create(
        version=version,
        input={
                **params,
                "prompt": template.format(completion=data["prompt"].strip()),
            },
    )
    predictions.append(prediction)

# Manage status

In [105]:
all_succeeded = False

while not all_succeeded:
    all_succeeded = True
    for prediction in predictions:
        if prediction.status == "succeeded":
            continue # Already succeeded, skip

        prediction.reload()
        if prediction.status == "succeeded": # succeeded임에도 계속 running으로 떠서 비용이 부과되는 문제가 있음. 수동으로 cancel
            prediction.cancel()
        else: 
            print(f"{prediction.id}: {prediction.status}")
            all_succeeded = False
        
        print(f"{prediction.id}: {prediction.status}")
    print("-----")

poio6cdbhkrbjim6tepv5jovgi: succeeded
qtrs2ulbhcpqawhx5ozd56fd6y: succeeded
n54lcwlbbwopprcerpd5v6y6x4: succeeded
3fvrxytbgay4uv23bu4ikzyglu: succeeded
g4q5wdtblrbuviuwyenalomzc4: succeeded
yzzxdplbar3zdnw766xo7d4nny: succeeded
4c4neltbreoit5zt3uo5oo4vpm: succeeded
uuzj6clbhlnne2ia4ufx3nnt3u: succeeded
4nxszglb42umsbzpr2hmaoe77y: succeeded
seuzemlbtfei3orsi7xkgmelg4: processing
seuzemlbtfei3orsi7xkgmelg4: processing
vzs6trlbsevrzkdywzp37jizci: starting
vzs6trlbsevrzkdywzp37jizci: starting
mowvujdbnhsgjpa27ac3kkyr4i: starting
mowvujdbnhsgjpa27ac3kkyr4i: starting
b7mbtptba6j3xgcrcih2jdtv7a: starting
b7mbtptba6j3xgcrcih2jdtv7a: starting
asvi4qlbweej4x2y22ygsk6cry: starting
asvi4qlbweej4x2y22ygsk6cry: starting
hvuorptbtc7bhoj3v72zl73p54: starting
hvuorptbtc7bhoj3v72zl73p54: starting
idqilzdbapv7277eu6j25oc6xq: starting
idqilzdbapv7277eu6j25oc6xq: starting
p3oclhlbsvglvi2s5mrvgmkawq: starting
p3oclhlbsvglvi2s5mrvgmkawq: starting
b3z64m3bda3wpykz6gfmwqr4gm: starting
b3z64m3bda3wpykz6gfmwqr4g

# Convert to JSON

In [106]:
for prediction in predictions:
    prediction.reload()
    prediction.cancel()

In [107]:
predictions

[Prediction(id='poio6cdbhkrbjim6tepv5jovgi', model='meta/codellama-34b-instruct', version='b17fdb44c843000741367ae3d73e2bb710d7428a662238ddebbf4302db2b5422', status='succeeded', input={'top_k': 50, 'top_p': 1, 'prompt': 'Your task is \nto condense Python function that must be between the [PYTHON] and [/PYTHON] tag.\nDo not change the function name. Please do not repeat.\nYou should apply following tips:\n1. Erase comments.\n2. Use dictionaries.\n3. Change for statements to comprehensions.\n4. Change if statements to conditional expressions or comprehensions.\n5. Use built-in functions.\n6. Use short circuit.\n7. Use "join" to print everything at once.\n8. Use short variable names.\nproblem :\n[PYTHON]\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i+1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) <= threshold:\n                return True\n    return False\n[/PYTHON]\n', 'max_tokens': 30

In [109]:
results = []
for i, prediction in enumerate(predictions):
    results.append({
        "task_id": dataset["task_id"][i],
        "completion": "".join(prediction.output)
    })
results

[{'task_id': 'HumanEval/0',
  'completion': '  \n[PYTHON]\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i, n1 in enumerate(numbers):\n        for n2 in numbers[i+1:]:\n            if abs(n1 - n2) <= threshold:\n                return True\n    return False\n[/PYTHON]\n\nThis code uses a more concise way to iterate over the elements of the list using the `enumerate` function, and the slicing notation to iterate over the remaining elements. It also uses the short-circuit evaluation of the `or` operator to simplify the logic.'},
 {'task_id': 'HumanEval/1',
  'completion': '  \n[PYTHON]\ndef separate_paren_groups(s):\n    open_brace = 0\n    start_index = 0\n    result = []\n    for i, c in enumerate(s):\n        if c == \'(\':\n            if open_brace == 0:\n                start_index = i\n            open_brace += 1\n        elif c == \')\':\n            open_brace -= 1\n            if open_brace == 0:\n                result.append(s[start_index:i+

In [110]:
with jsonlines.open("raw.jsonl", mode="w") as writer:
    for result in results:
        writer.write(result)

In [ ]:
for result in results:
    # STrings are wrapped with [PYTHON] [/PYTHON] tags. Remove them.
    try:
        assert "```" in result["completion"]
    except AssertionError:
        print(result["task_id"])
        # result["completion"] = result["completion"].strip()
        print(result["completion"])
        print("-" * 50)
        continue
    if "```" in result["completion"]:
        result["completion"] = result["completion"].split("```")[1].split("```")[0].strip()

results

In [ ]:
for result in results:
    # STrings are wrapped with [PYTHON] [/PYTHON] tags. Remove them.
    try:
        assert "[PYTHON]" in result["completion"]
    except AssertionError:
        print(result["task_id"])
        # result["completion"] = result["completion"].strip()
        print(result["completion"])
        print("-" * 50)
        continue
    if "[PYTHON]" in result["completion"]:
        result["completion"] = result["completion"].split("[PYTHON]")[1].split("[/PYTHON]")[0].strip()

results

In [ ]:
for result in results:
    if not result["completion"].startswith("def"):
        result["completion"] = result["completion"].replace("python\n", "")
        print(result["completion"])

In [ ]:
import re
from pprint import pprint

for result in results:
    # STrings are wrapped with [PYTHON] [/PYTHON] tags. Remove them.
    try:
        assert "1." in result["completion"]
    except AssertionError as e:
        print(result["task_id"])
        print(result["completion"])
        result["completion"] = "" # Set to empty string if no plan is found.
        continue
    
    lines = result["completion"].split("\n")
    # Only keep the lines that matches regex "\d."
    lines = [line for line in lines if re.match(".*\.", line[:5])]
    result["completion"] = "\n".join(lines)
    result["completion"] = result["completion"].strip()
results

In [112]:
for result in results:
    print(result["completion"])
    print("-" * 80)

def has_close_elements(numbers: List[float], threshold: float) -> bool:
    for i, n1 in enumerate(numbers):
        for n2 in numbers[i+1:]:
            if abs(n1 - n2) <= threshold:
                return True
    return False
--------------------------------------------------------------------------------
def separate_paren_groups(s):
    open_brace = 0
    start_index = 0
    result = []
    for i, c in enumerate(s):
        if c == '(':
            if open_brace == 0:
                start_index = i
            open_brace += 1
        elif c == ')':
            open_brace -= 1
            if open_brace == 0:
                result.append(s[start_index:i+1])
    return result
--------------------------------------------------------------------------------
def truncate_number(number: float) -> float:
    return number % 1
--------------------------------------------------------------------------------
def below_zero(ops: List[int]) -> bool:
    return any(balance < 0 for balance in 

In [66]:
results[157]["completion"] = """def right_angle_triangle(a, b, c):
    # Calculate the lengths of the sides using the Pythagorean theorem
    hypotenuse = (a**2 + b**2)**0.5
    
    # Check if the length of the hypotenuse is equal to the sum of the lengths of the other two sides
    if hypotenuse == c:
        return True
    else:
        return False
"""

print(results[157]["completion"])

def right_angle_triangle(a, b, c):
    # Calculate the lengths of the sides using the Pythagorean theorem
    hypotenuse = (a**2 + b**2)**0.5
    
    # Check if the length of the hypotenuse is equal to the sum of the lengths of the other two sides
    if hypotenuse == c:
        return True
    else:
        return False



In [113]:
experiment_name = "summary_2_reasoning"
path_json = f"predictions/prediction_{experiment_name}.jsonl"
path_txt = f"predictions/prediction_{experiment_name}.txt"

In [114]:
with jsonlines.open(path_json, mode="w") as writer:
    for result in results:
        writer.write(result)

In [115]:
with open(path_txt, "w") as writer:
    for result in results:
        writer.write(f"{result['task_id']}\n\n")
        writer.write(f"{result['completion']}\n\n")
        writer.write('-'*50 + '\n')

In [116]:
program = ""
with open('./t.txt', "r") as f:
    program = f.read()

['from typing import List\n', '\n', '\n', 'def has_close_elements(numbers: List[float], threshold: float) -> bool:\n', '    """ Check if in given list of numbers, are any two numbers closer to each other than\n', '    given threshold.\n', '    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n', '    False\n', '    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n', '    True\n', '    """\n', '\n', 'def has_close_elements(numbers: List[float], threshold: float) -> bool:\n', '    for i, n1 in enumerate(numbers):\n', '        for n2 in numbers[i+1:]:\n', '            if abs(n1 - n2) <= threshold:\n', '                return True\n', '    return False\n', 'def check(candidate):\n', '    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n', 'check(has_close_elements)']


# Emergency Cancel

In [ ]:
# 실수했을 때 모든 prediction을 취소 (돈 아까워서)
for prediction in predictions:
    prediction.cancel()

In [417]:
preds = []
with open("predictions/prediction_summary_2_1shot_2.txt") as reader:
    full = "".join(reader.readlines())
full

'HumanEval/0\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    return any(abs(a - b) <= threshold for a, b in zip(numbers, numbers[1:]))\n\n--------------------------------------------------\nHumanEval/1\n\ndef separate_paren_groups(paren_string):\n    return re.findall(r\'\\(.*?\\)\', paren_string)\n\n--------------------------------------------------\nHumanEval/2\n\ndef truncate_number(number: float) -> float:\n    return number % 1\n\n--------------------------------------------------\nHumanEval/3\n\ndef below_zero(operations: List[int]) -> bool:\n    return any(operation < 0 for operation in operations)\n\n--------------------------------------------------\nHumanEval/4\n\ndef minLengthAfterRemovals(self, nums: List[int]) -> int:\n    scount = Counter(nums)\n    arr = sorted([scount[num] for num in sorted(scount.keys())], key=lambda x: -x)    \n    \n    return (arr[0] - sum(arr[1:])) if arr[0] > sum(arr[1:]) else sum(arr) % 2\n\n-----------------------

In [424]:
preds = full.split("-"*50)[:-1]
preds

['HumanEval/0\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    return any(abs(a - b) <= threshold for a, b in zip(numbers, numbers[1:]))\n\n',
 "\nHumanEval/1\n\ndef separate_paren_groups(paren_string):\n    return re.findall(r'\\(.*?\\)', paren_string)\n\n",
 '\nHumanEval/2\n\ndef truncate_number(number: float) -> float:\n    return number % 1\n\n',
 '\nHumanEval/3\n\ndef below_zero(operations: List[int]) -> bool:\n    return any(operation < 0 for operation in operations)\n\n',
 '\nHumanEval/4\n\ndef minLengthAfterRemovals(self, nums: List[int]) -> int:\n    scount = Counter(nums)\n    arr = sorted([scount[num] for num in sorted(scount.keys())], key=lambda x: -x)    \n    \n    return (arr[0] - sum(arr[1:])) if arr[0] > sum(arr[1:]) else sum(arr) % 2\n\n',
 '\nHumanEval/5\n\ndef intersperse(numbers: List[int], delimeter: int) -> List[int]:\n    return [x for pair in zip(numbers[::2], numbers[1::2]) for x in pair] + numbers[-1:]\n\n',
 "\nHumanEval/6\n\nd

In [426]:
type(preds)

list

In [429]:
for i, pred in enumerate(preds):
    preds[i] = re.sub("HumanEval.*\n", "", pred)
    preds[i] = preds[i].strip()
preds

['def has_close_elements(numbers: List[float], threshold: float) -> bool:\n    return any(abs(a - b) <= threshold for a, b in zip(numbers, numbers[1:]))',
 "def separate_paren_groups(paren_string):\n    return re.findall(r'\\(.*?\\)', paren_string)",
 'def truncate_number(number: float) -> float:\n    return number % 1',
 'def below_zero(operations: List[int]) -> bool:\n    return any(operation < 0 for operation in operations)',
 'def minLengthAfterRemovals(self, nums: List[int]) -> int:\n    scount = Counter(nums)\n    arr = sorted([scount[num] for num in sorted(scount.keys())], key=lambda x: -x)    \n    \n    return (arr[0] - sum(arr[1:])) if arr[0] > sum(arr[1:]) else sum(arr) % 2',
 'def intersperse(numbers: List[int], delimeter: int) -> List[int]:\n    return [x for pair in zip(numbers[::2], numbers[1::2]) for x in pair] + numbers[-1:]',
 "def parse_nested_parens(paren_string: str) -> List[int]:\n    def get_max_level(s: str) -> int:\n        level = 0\n        max_level = 0\n   

In [430]:
with jsonlines.open("predictions/prediction_summary_2_1shot_2.jsonl", mode="w") as writer:
    for i, pred in enumerate(preds):
        writer.write({"task_id": dataset[i]["task_id"], "completion": pred})